In [31]:
import spacy
import nltk
import statistics as s
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
import numpy as np
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification




nlp = spacy.load("en_core_web_md")
nltk.download('punkt')


model = SentenceTransformer('all-MiniLM-L6-v2')
classifier = pipeline("sentiment-analysis")
summarizer = pipeline("summarization")
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model_token = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
token = pipeline("ner", model=model_token, tokenizer=tokenizer)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nachikethinge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


# Collecting Articles

In [262]:
#About Ukraine (1)
url1 = 'https://www.foxnews.com/us/sister-us-citizen-killed-ukraine-recounts-final-days' 
article1 = Article(url1)
article1.download()
article1.parse()
article1.text

'NEW You can now listen to Fox News articles!\n\nKatya Hill, sister of slain U.S. citizen Jimmy Hill, spoke to reporters Saturday about her brother\'s death in Ukraine.\n\nJimmy Hill was reported dead Wednesday morning after a Russian bomb exploded in the city of Chernihiv. Jimmy Hill was in Ukraine at the time to help his partner get critical health care procedures when he was reportedly killed in a breadline. But Katya Hill updated the press on new discoveries in the circumstances of her brother\'s death.\n\n"Although initial reports indicated that he was killed in a breadline, the State Department informed us that his death was from a Russian bomb. Jimmy was in a civilian area of the city near the Chernihiv hospital. The State Department has not yet contacted the family to let us know the specifics," Hill told the press.\n\nAMERICAN KILLED BY RUSSIAN FORCES IN UKRAINE A \'HELPER, A PEACEMAKER\': SISTER SAYS\n\n"My brother was in Chernihiv with the purpose of organizing medical treat

In [127]:
#About Ukraine (2)
url2 = 'https://www.cbsnews.com/news/james-hill-us-citizen-killed-ukraine/' 
article2 = Article(url2)
article2.download()
article2.parse()

In [128]:
#About March Madness
url3 = 'https://longhornswire.usatoday.com/lists/texas-basketball-big-12-ncaa-tournament-power-conferences-first-round-results/'
article3 = Article(url3)
article3.download()
article3.parse()

# Creating Summary of the articles

In [ ]:
#article1.nlp()
#summary1 = article1.summary
#class1 = nlp(summary1)
#sentences1 = str(list(class1.sents))
#sentences1

In [234]:
summary1 = list(summarizer(article1.text, max_length = 130, min_length = 30)[0].values())[0]
summary1

In [239]:
summary2 = list(summarizer(article2.text, max_length = 130, min_length = 30)[0].values())[0]
summary2

' "My brother was a giver, and he just felt everybody in the world should love me" "My parents raised us to see the world would let this happen, and that\'s how he lived" "A good person gone too soon" has not decided whether the public will "want to hear" or "don\'t feel that the public is going to be going to the end of the year" "I\'m not going to hear that we didn\'t want the public to hear," said the public should "have no way of hearing" The public will be able to decide whether or not to hear the public\'s view of the'

In [238]:
summary3 = list(summarizer(article3.text, max_length = 130, min_length = 30)[0].values())[0]
summary3

' It has been just two days of the March Madness tournament and there have been multiple upsets already . The Big 12 did well, as all six teams that made the big dance were able to survive and advance . Here is how the Power 6 conferences fared .'

In [240]:
type(summary3)

str

# Similarities of Articles 

In [183]:
#1st Article: Ukraine.. 2nd Article: Ukraine SIMILARITY
emb1 = model.encode(summary1)
emb2 = model.encode(summary2)
cos_sim12 = util.cos_sim(emb1, emb2).numpy()[0][0]
print(f"",cos_sim12 * 100, "%")


 81.91240429878235 %


In [184]:
#1st Article: Ukraine.. 3RD Article: MARCH MADNESS SIMILARITY
emb1 = model.encode(summary1)
emb3 = model.encode(summary3)
cos_sim13 = util.cos_sim(emb1, emb3).numpy()[0][0]
print(f"",cos_sim13 * 100, "%")

 1.989033818244934 %


In [185]:
#2nd Article: Ukraine.. 3RD Article: MARCH MADNESS SIMILARITY
emb2 = model.encode(summary2)
emb3 = model.encode(summary3)
cos_sim23 = util.cos_sim(emb2, emb3).numpy()[0][0]
print(f"",cos_sim23 * 100, "%")

 3.5989344120025635 %


# Positive and Negative score of an article

In [237]:
classifier(summary1)

[{'label': 'NEGATIVE', 'score': 0.9888350367546082}]

In [200]:
classifier(article2.summary)

[{'label': 'NEGATIVE', 'score': 0.9913444519042969}]

In [201]:
classifier(article3.summary)

[{'label': 'POSITIVE', 'score': 0.9779484272003174}]

In [23]:
#Getting everything together
def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

def user_summarize(article):
    summary = list(summarizer(article)[0].values())[0]
    return summary



In [26]:
print("Welcome!\n")
user_url = input("Enter the url of the article to evaluate: ")
print("\nGetting the information...\n")
article = get_article(str(user_url))
summary = user_summarize(article)
result = classifier(summary)
print("Here is your information: \n")
print("Summary: ", summary)
print("")
print(f"The text is labeled as: {list(result[0].values())[0]} with a score of {list(result[0].values())[1]}")


Welcome!

Enter the url of the article to evaluate: https://thehill.com/changing-america/enrichment/arts-culture/598832-james-webb-telescope-sends-back-new-image-of-star

Getting the information...

Here is your information: 

Summary:   NASA’s new James Webb telescope has reached another alignment milestone and sent back its clearest image of a star to date . The Webb Space Telescope launched from French Guiana on Christmas Day to replace the aging Hubble telescope . NASA stressed that the telescope will not be able to capture images as designed for several more months .

The text is labeled as: POSITIVE with a score of 0.6808596253395081


In [52]:
list(token(summary)[].values())[3]

'James'

In [53]:
token(summary)

[{'entity': 'B-ORG',
  'score': 0.9989575,
  'index': 1,
  'word': 'NASA',
  'start': 1,
  'end': 5},
 {'entity': 'B-PER',
  'score': 0.9675067,
  'index': 5,
  'word': 'James',
  'start': 12,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.76386505,
  'index': 6,
  'word': 'Webb',
  'start': 18,
  'end': 22},
 {'entity': 'B-ORG',
  'score': 0.98394287,
  'index': 27,
  'word': 'Webb',
  'start': 130,
  'end': 134},
 {'entity': 'I-ORG',
  'score': 0.9885953,
  'index': 28,
  'word': 'Space',
  'start': 135,
  'end': 140},
 {'entity': 'I-ORG',
  'score': 0.98297256,
  'index': 29,
  'word': 'Tel',
  'start': 141,
  'end': 144},
 {'entity': 'I-ORG',
  'score': 0.9603502,
  'index': 30,
  'word': '##es',
  'start': 144,
  'end': 146},
 {'entity': 'I-ORG',
  'score': 0.90703136,
  'index': 31,
  'word': '##cope',
  'start': 146,
  'end': 150},
 {'entity': 'B-LOC',
  'score': 0.9995146,
  'index': 34,
  'word': 'French',
  'start': 165,
  'end': 171},
 {'entity': 'I-LOC',
  'score': 0.99926